In [1]:
import FPCAnalysis
import numpy as np

In [2]:
#load everything we need
#-----------------------

path = '/home/crbrown/M06_th45/'
numframe = 2000
inputs = FPCAnalysis.ddhr.read_input(path=path)

#load the fields data
dfields = FPCAnalysis.ddhr.field_loader(path=path,num=numframe)
dfields.keys()

#load the density data
dden = FPCAnalysis.ddhr.den_loader(path=path,num=numframe)
dden.keys()

#load subset of particles that we want to compute the FPC of
path_particles = path+"Output/Raw/Sp01/raw_sp01_{:08d}.h5" #note that this specifies the species!
x1 = 39.5
x2 = 41.5
y1 = 0
y2 = .5 #note, for the example data, yz plane is 12x12 d_i large, but here we load just part of that for computational efficiency while learning
z1 = 0
z2 = .5
dparticles = FPCAnalysis.ddhr.read_box_of_particles(path_particles, numframe, x1, x2, y1, y2, z1, z2)



In [3]:
#Note, this analysis is performed in the simulation rest frame!!!

In [4]:
#compute dfluc
# At the core of our shock analysis is the idea that we can separate the fields into fluctuating and steady state fields

dfluc = FPCAnalysis.anl.remove_average_fields_over_yz(dfields)
dfavg = FPCAnalysis.anl.get_average_fields_over_yz(dfields)

testkey = 'ex'
testidxx = 12
testidyy = 5
testidzz = 35

#these should be equal
dfluc[testkey][testidzz,testidyy,testidxx]+dfavg[testkey][testidzz,testidyy,testidxx],dfields[testkey][testidzz,testidyy,testidxx]

In [5]:
print("For example, E_x = \overline{E_x} + \widetilde{E_x}")

In [6]:
import matplotlib.pyplot as plt

#we typically work with the electric fields, 
#  but it's easier to understand if we look at the compressible component of the magnetic field

plt.figure(figsize=(10,3))
plt.plot(dfields['ex_xx'],dfields['bz'][0,0,:],label=r'$B_z(z_0,y_0,x)$')
plt.plot(dfields['ex_xx'],dfluc['bz'][0,0,:],label=r'$\widetilde{B_z}(z_0,y_0,x)$')
plt.plot(dfields['ex_xx'],dfavg['bz'][0,0,:],label=r'$\overline{B_z}(z_0,y_0,x)$')
plt.grid()
plt.legend()
plt.show()

In [7]:
#Do the wavelet transform of the fluctuating fields
#--------------------------------------------------

lowerresolution = True #lower field resolution for speed!
if(lowerresolution):
    print('dfields["ex"].shape before:',dfields['ex'].shape)
    dfields = FPCAnalysis.ao.avg_dict(dfields,binidxsz=[2,2,2]) #note, each element in binidxsz must be an integer divisor of the shape of its respective axis
    dfluc = FPCAnalysis.ao.avg_dict(dfluc,binidxsz=[2,2,2])
    dfavg = FPCAnalysis.ao.avg_dict(dfavg,binidxsz=[2,2,2])
    print('dfields["ex"].shape after:',dfields['ex'].shape)

In [8]:
#computes ex(kz,ky,kz;xx)
kz, ky, kx, exkzkykxxx = FPCAnalysis.anl.transform_field_to_kzkykxxx(dfields,'ex')
xx = dfields['ex_xx']

In [9]:
kz.shape,ky.shape,kx.shape,xx.shape,exkzkykxxx.shape

In [10]:
kzidx = 0
kyidx = 0

print('kz,ky of this plot is ',kz[kzidx],ky[kyidx])

#vars that make model shock on bottom panel
Bz2_over_Bz1 = 3.8 #approx compression ratio
xpos_shock = 38.625 #position of jump of model shock
xpos_line = 39.875 #vertical black line

fieldvals = dfavg['bz'][0,0,:]

flnm = ''
FPCAnalysis.pltfr.plot_wlt_over_field(xx, kx, exkzkykxxx[kzidx,kyidx,::], fieldvals, flnm = flnm, ylim = [0,5],
               Bz2_over_Bz1 = Bz2_over_Bz1, xpos_shock = xpos_shock, xpos_line = xpos_line)